In [ ]:
import pandas as pd
all_data_price = pd.read_pickle("all_data_price.pkl")
# all_data_price.loc[0,'data']
#all_data_price.set_index("stock", inplace = True)


ri_history = pd.read_pickle("ri_history.pkl")
dataRi=ri_history
dataRi.set_index('stock')
ri_history.loc[0,'data']

MarketWatch = pd.read_pickle("MarketWatch.pkl")
MarketWatch
all_data_price

In [ ]:
def min_price(n):
        minprice=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                s=(all_data_price.loc[i,'data'])['Low'].iloc[-n].copy()
                m=0
                for j in range(n+2):
                    if j==0 or j==1:
                        continue

                    m=(all_data_price.loc[i,'data'])['Low'].iloc[-j].copy()
                    if m<s:
                        s=m
                if s==0:
                    continue
                print(s)
                minprice.at[i,'data']=s.copy()
            except Exception as err:
                print('\n no %s for: ' % (n),i)
        return minprice
g=min_price(60)
g#.loc['آبادا','data']

# gold filter

In [ ]:
# def acc_volume(n):
#         volume=all_data_price.copy()
#         stock=list(all_data_price.index)
#         for i in stock:
#             try:
#                 s=0
#                 m=0
#                 for j in range(n+2):
#                     if j==0 or j==1:
#                         continue

#                     m=(all_data_price.loc[i,'data'])['Volume'].iloc[-j]
#                     s=m+s
#                 volume.loc[i,'data']=int(s/n)
#             except Exception as err:
#                 print('\n no volume for: ',i)

#         return volume
# def min_price(n):
#         minprice=save_filters.copy()
#         stock=bourse_stocks
#         for i in stock:
#             try:
#                 s=(all_data_price.loc[i,'data'])['Low'].iloc[-n].copy()
#                 m=0
#                 for j in range(n+2):
#                     if j==0 or j==1:
#                         continue

#                     m=(all_data_price.loc[i,'data'])['Low'].iloc[-j].copy()
#                     if m<s:
#                         s=m
#                 if s==0:
#                     continue
#                 minprice.loc[i,'data']=s
#             except Exception as err:
#                 print('\n no %s for: ' % (n),i)

#         return minprice
# def I_to_R_exit_money3():
#     volume365=acc_volume(365)
#     I_to_R_exit_money3=save_filters.copy()
#     stock=list(all_data_price.index)
#     for i in stock:
#         try:
#             I_to_R_exit_money3.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume365.loc[i,'data']) and \
#             (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
#             ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
#             (MarketWatch.loc[i,'Vol_Sell_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
#             (MarketWatch.loc[i,'Vol_Buy_I'])>0.5*(MarketWatch.loc[i,'Volume'])

#         except Exception as err:
#             print(i,end='-')
#     return I_to_R_exit_money3
# I_to_R_exit_money3()

In [ ]:
#     در بورس <<<<  I = حقیقی
#     در بورس <<<<  N = حقوقی

# در دیتا فریم<<<<< I = حقوقی
# در دیتا فریم<<<<< R = حقیقی

#Final=pc , close=pl
import pandas as pd
from persiantools.jdatetime import JalaliDate
import datetime
import time
import os.path
import numpy as np
import finpy_tse as fpy

start_time=time.perf_counter()


df_stock_list = fpy.Build_Market_StockList(bourse = True, farabourse = False, payeh = False,
detailed_list = True, show_progress = True,save_excel = False, save_csv = False)
bourse_stocks = df_stock_list.index.tolist()

MarketWatch = pd.read_pickle("MarketWatch.pkl")
all_data_price = pd.read_pickle("all_data_price.pkl")
save_filters = pd.DataFrame({'stock': bourse_stocks, 'data': bourse_stocks})
save_filters = save_filters.set_index('stock')

class filters:
    
    def acc_volume(self,n):
        volume=all_data_price.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                s=0
                m=0
                for j in range(n+2):
                    if j==0 or j==1:
                        continue

                    m=(all_data_price.loc[i,'data'])['Volume'].iloc[-j]
                    s=m+s
                volume.loc[i,'data']=int(s/n)
            except Exception as err:
                print('\n no volume for: ',i)

        return volume

    def pcc_plc(self):
        stock=list(all_data_price.index)
        pcc_plc = pd.DataFrame({'stock': stock, 'pcc': stock, 'plc':stock})
        pcc_plc = save_filters.copy()
        for i in bourse_stocks:
            pcc_plc.loc[i,'plc']=(all_data_price.loc[i,'data'])['Final'].iloc[-1]-(all_data_price.loc[i,'data'])['Final'].iloc[-2]
            pcc_plc.loc[i,'pcc']=(all_data_price.loc[i,'data'])['Close'].iloc[-1]-(all_data_price.loc[i,'data'])['Final'].iloc[-2]
        return pcc_plc

    def seri_final(self,i,n):
        for j in range(n):
            t=(all_data_price.loc[i,'data'])['Final'].iloc[-n-1]<0.97*(all_data_price.loc[i,'data'])['Final'].iloc[-n-2]
            if t==False:
                break
        return t

    def minprice(self,n):
        minprice=save_filters.copy()
        stock=bourse_stocks
        for i in stock:
            try:
                s=(all_data_price.loc[i,'data'])['Low'].iloc[-n].copy()
                m=0
                for j in range(n+2):
                    if j==0 or j==1:
                        continue

                    m=(all_data_price.loc[i,'data'])['Low'].iloc[-j]
                    if m<s:
                        s=m
                if s==0:
                    continue
                minprice.loc[i,'data']=s
            except Exception as err:
                print('\n no %s for: ' % (n),i)

        return minprice

    def sort_marketcap(self):
        sort_marketcap=all_data_price.copy()
        stock=list(all_data_price.index)
        stop_stocks=[]
        for i in stock:
            try:
                sort_marketcap.loc[i,'data']=MarketWatch.loc[i,'Market Cap']
            except Exception as err:
                stop_stocks.append(i)
                sort_marketcap.loc[i,'data']=0
        dive=sort_marketcap['data'].max()
        sort_marketcap['data'] = sort_marketcap['data'].div(dive)
        return sort_marketcap

    def p_E(self):
        p_e=all_data_price.copy()
        stock=list(all_data_price.index)
        stop_stocks=[]
        for i in stock:
            try:
                p_e.loc[i,'data']=(MarketWatch.loc[i,'Final'])/(MarketWatch.loc[i,'EPS'])
            except Exception as err:
                stop_stocks.append(i)
                p_e.loc[i,'data']=0
        p_e=p_e.sort_values(by=['data'], ascending=False)
        p_e[p_e['data'] == np.inf] = 0
        return p_e  
   
    def smart_enter_money1(self):
        smart_enter_money1=save_filters.copy()
        volume30=self.acc_volume(30)
        stock=list(all_data_price.index)
        for i in stock:
            try:
                smart_enter_money1.at[i,'data']=(MarketWatch.loc[i,'Volume']>volume30.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0)
            except Exception as err:
                print(i,end='-')
        return smart_enter_money1

    def smart_enter_money2(self):
        volume90=self.acc_volume(90)
        smart_enter_money2=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                smart_enter_money2.loc[i,'data']=(MarketWatch.loc[i,'Volume']>1.25*volume90.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0)

            except Exception as err:
                print(i,end='-')
        return smart_enter_money2

    def smart_enter_money3(self):
        volume365=self.acc_volume(365)
        smart_enter_money3=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                smart_enter_money3.loc[i,'data']=(MarketWatch.loc[i,'Volume']>1.5*volume365.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0)

            except Exception as err:
                print(i,end='-')
        return smart_enter_money3

    def I_to_R_smart_enter_money4(self):
        volume30=self.acc_volume(30)
        I_to_R_smart_enter_money4=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_smart_enter_money4.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume30.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_smart_enter_money4

    def I_to_R_smart_enter_money5(self):
        volume90=self.acc_volume(90)
        I_to_R_smart_enter_money5=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_smart_enter_money5.loc[i,'data']=(MarketWatch.loc[i,'Volume']>1.25*volume90.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_smart_enter_money5

    def I_to_R_smart_enter_money6(self):
        volume365=self.acc_volume(365)
        I_to_R_smart_enter_money6=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_smart_enter_money6.loc[i,'data']=(MarketWatch.loc[i,'Volume']>1.5*volume365.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_smart_enter_money6

    def hot_money1(self):
        volume30=self.acc_volume(30)
        hot_money1=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                hot_money1.loc[i,'data']=(MarketWatch.loc[i,'Volume']>2*volume30.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                2*((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0)

            except Exception as err:
                print(i,end='-')
        return hot_money1

    def hot_money2(self):
        volume90=self.acc_volume(90)
        hot_money2=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                hot_money2.loc[i,'data']=(MarketWatch.loc[i,'Volume']>3*volume90.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                2*((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0)

            except Exception as err:
                print(i,end='-')
        return hot_money2

    def hot_money3(self):
        volume365=self.acc_volume(365)
        hot_money3=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                hot_money3.loc[i,'data']=(MarketWatch.loc[i,'Volume']>4*volume365.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                2*((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0)

            except Exception as err:
                print(i,end='-')
        return hot_money3

    def I_to_R_hot_money1(self):
        volume30=self.acc_volume(30)
        I_to_R_hot_money1=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_hot_money1.loc[i,'data']=(MarketWatch.loc[i,'Volume']>2*volume30.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                2*((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_hot_money1

    def I_to_R_hot_money2(self):
        volume90=self.acc_volume(90)
        I_to_R_hot_money2=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_hot_money2.loc[i,'data']=(MarketWatch.loc[i,'Volume']>3*volume90.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                2*((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_hot_money2

    def I_to_R_hot_money3(self):
        volume365=self.acc_volume(365)
        I_to_R_hot_money3=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_hot_money3.loc[i,'data']=(MarketWatch.loc[i,'Volume']>4*volume365.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                2*((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])>=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])>0) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_hot_money3

    def enter_money1(self):
        volume30=self.acc_volume(30)
        enter_money1=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                enter_money1.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume30.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']))

            except Exception as err:
                print(i,end='-')
        return enter_money1

    def enter_money2(self):
        volume90=self.acc_volume(90)
        enter_money2=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                enter_money2.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume90.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']))

            except Exception as err:
                print(i,end='-')
        return enter_money2

    def enter_money3(self):
        volume365=self.acc_volume(365)
        enter_money3=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                enter_money3.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume365.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']))

            except Exception as err:
                print(i,end='-')
        return enter_money3

    def I_to_R_enter_money1(self):
        volume30=self.acc_volume(30)
        I_to_R_hot_money1=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_hot_money1.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume30.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>= \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_hot_money1

    def I_to_R_enter_money2(self):
        volume90=self.acc_volume(90)
        I_to_R_hot_money2=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_hot_money2.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume90.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>= \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_hot_money2

    def I_to_R_enter_money3(self):
        volume365=self.acc_volume(365)
        I_to_R_hot_money3=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_hot_money3.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume365.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>= \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_hot_money3

    def I_to_R_vbR_vsI_greater50_bR_greater_sR(self):
        I_to_R_vbR_vsI_greater50_bR_greater_sR=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_vbR_vsI_greater50_bR_greater_sR.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])> \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_vbR_vsI_greater50_bR_greater_sR

    def I_to_R_vbR_vsI_greater70(self):
        I_to_R_vbR_vsI_greater70=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_vbR_vsI_greater70.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_R'])>0.7*MarketWatch.loc[i,'Volume'] and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.7*MarketWatch.loc[i,'Volume']

            except Exception as err:
                print(i,end='-')
        return I_to_R_vbR_vsI_greater70

    def NbI_greater_2NsI_and_bR_greater_sR(self):
        NbI_greater_2NsI_and_bR_greater_sR=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                NbI_greater_2NsI_and_bR_greater_sR.loc[i,'data']=(MarketWatch.loc[i,'No_Buy_I'])>=2*MarketWatch.loc[i,'No_Sell_I'] and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])> \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']))

            except Exception as err:
                print(i,end='-')
        return NbI_greater_2NsI_and_bR_greater_sR

    def smart_exit_money1(self):
        volume30=self.acc_volume(30)
        smart_exit_money1=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                smart_exit_money1.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume30.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])<=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])<0)

            except Exception as err:
                print(i,end='-')
        return smart_exit_money1

    def smart_exit_money2(self):
        volume90=self.acc_volume(90)
        smart_exit_money2=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                smart_exit_money2.loc[i,'data']=(MarketWatch.loc[i,'Volume']>1.25*volume90.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])<=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])<0)

            except Exception as err:
                print(i,end='-')
        return smart_exit_money2

    def smart_exit_money3(self):
        volume365=self.acc_volume(365)
        smart_exit_money3=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                smart_exit_money3.loc[i,'data']=(MarketWatch.loc[i,'Volume']>1.5*volume365.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])<=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])<0)

            except Exception as err:
                print(i,end='-')
        return smart_exit_money3

    def I_to_R_smart_exit_money1(self):
        volume30=self.acc_volume(30)
        I_to_R_smart_exit_money1=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_smart_exit_money1.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume30.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])<=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])<0) and \
                (MarketWatch.loc[i,'Vol_Sell_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Buy_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_smart_exit_money1

    def I_to_R_smart_exit_money2(self):
        volume90=self.acc_volume(90)
        I_to_R_smart_exit_money2=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_smart_exit_money2.loc[i,'data']=(MarketWatch.loc[i,'Volume']>1.25*volume90.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])<=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])<0) and \
                (MarketWatch.loc[i,'Vol_Sell_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Buy_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_smart_exit_money2

    def I_to_R_smart_exit_money3(self):
        volume365=self.acc_volume(365)
        I_to_R_smart_exit_money3=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_smart_exit_money3.loc[i,'data']=(MarketWatch.loc[i,'Volume']>1.5*volume365.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Close'])<=(MarketWatch.loc[i,'Final']) and ((MarketWatch.loc[i,'Close(%)'])<0) and \
                (MarketWatch.loc[i,'Vol_Sell_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Buy_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_smart_exit_money3

    def exit_money1(self):
        volume30=self.acc_volume(30)
        exit_money1=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                exit_money1.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume30.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']))

            except Exception as err:
                print(i,end='-')
        return exit_money1

    def exit_money2(self):
        volume90=self.acc_volume(90)
        exit_money2=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                exit_money2.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume90.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']))

            except Exception as err:
                print(i,end='-')
        return exit_money2

    def exit_money3(self):
        volume365=self.acc_volume(365)
        exit_money3=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                exit_money3.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume365.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']))

            except Exception as err:
                print(i,end='-')
        return exit_money3

    def I_to_R_exit_money1(self):
        volume30=self.acc_volume(30)
        I_to_R_exit_money1=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_exit_money1.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume30.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Vol_Sell_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Buy_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_exit_money1

    def I_to_R_exit_money2(self):
        volume90=self.acc_volume(90)
        I_to_R_exit_money2=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_exit_money2.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume90.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Vol_Sell_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Buy_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_exit_money2

    def I_to_R_exit_money3(self):
        volume365=self.acc_volume(365)
        I_to_R_exit_money3=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                I_to_R_exit_money3.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume365.loc[i,'data']) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Vol_Sell_R'])>0.5*(MarketWatch.loc[i,'Volume']) and \
                (MarketWatch.loc[i,'Vol_Buy_I'])>0.5*(MarketWatch.loc[i,'Volume'])

            except Exception as err:
                print(i,end='-')
        return I_to_R_exit_money3

    def R_to_I_vbI_vsR_greater50_sR_greater_bR(self):
        R_to_I_vbI_vsR_greater50_sR_greater_bR=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                R_to_I_vbI_vsR_greater50_sR_greater_bR.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])) and \
                (MarketWatch.loc[i,'Vol_Buy_R'])>0.5*MarketWatch.loc[i,'Volume'] and \
                (MarketWatch.loc[i,'Vol_Sell_I'])>0.5*MarketWatch.loc[i,'Volume']

            except Exception as err:
                print(i,end='-')
        return R_to_I_vbI_vsR_greater50_sR_greater_bR

    def R_to_I_vbI_vsR_greater70(self):
        R_to_I_vbI_vsR_greater70=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                R_to_I_vbI_vsR_greater70.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_I'])>0.5*MarketWatch.loc[i,'Volume'] and \
                (MarketWatch.loc[i,'Vol_Sell_R'])>0.5*MarketWatch.loc[i,'Volume']

            except Exception as err:
                print(i,end='-')
        return R_to_I_vbI_vsR_greater70

    def NsI_greater_2NsI_and_sR_greater_bR(self):
        NsI_greater_2NsI_and_sR_greater_bR=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                NsI_greater_2NsI_and_sR_greater_bR.loc[i,'data']=(MarketWatch.loc[i,'No_Sell_I'])>=2*MarketWatch.loc[i,'No_Buy_I'] and \
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])< \
                ((MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']))

            except Exception as err:
                print(i,end='-')
        return NsI_greater_2NsI_and_sR_greater_bR

    def close_103finally_bR_greater_sR_clockp(self):
        close_103finally_bR_greater_sR_clockp=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                close_103finally_bR_greater_sR_clockp.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R']) > \
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']) and \
                (MarketWatch.loc[i,'Close'])>=1.03*(MarketWatch.loc[i,'Final'])

            except Exception as err:
                print(i,end='-')
        return close_103finally_bR_greater_sR_clockp

    def pclose_102pfinally_bR_greater_sR_clockp(self):
        pclose_102pfinally_bR_greater_sR_clockp=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                pclose_102pfinally_bR_greater_sR_clockp.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R']) > \
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']) and \
                ((MarketWatch.loc[i,'Close(%)']-MarketWatch.loc[i,'Final(%)'])>2) and \
                (MarketWatch.loc[i,'Low']==MarketWatch.loc[i,'Day_LL'])

            except Exception as err:
                print(i,end='-')
        return pclose_102pfinally_bR_greater_sR_clockp

    def close103_finally_sR_greater_bR_clockn(self):
        close103_finally_sR_greater_bR_clockn=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                close103_finally_sR_greater_bR_clockn.loc[i,'data']=\
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R']) < \
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']) and \
                (1.03*MarketWatch.loc[i,'Close'])<=(MarketWatch.loc[i,'Final'])

            except Exception as err:
                print(i,end='-')
        return close103_finally_sR_greater_bR_clockn

    def pclose102_pfinally_sR_greater_bR_clockn(selfself):
        pclose102_pfinally_sR_greater_bR_clockn=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                pclose102_pfinally_sR_greater_bR_clockn.loc[i,'data']=\
                (MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R']) < \
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R']) and \
                (MarketWatch.loc[i,'Final(%)']-(MarketWatch.loc[i,'Close(%)'])>2)
            except Exception as err:
                    print(i,end='-')
        return pclose102_pfinally_sR_greater_bR_clockn

    def skeptic_volume (self):
        volume30=self.acc_volume(30)
        skeptic_volume=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                skeptic_volume.loc[i,'data']=(MarketWatch.loc[i,'Volume']>5*volume30.loc[i,'data'])
            except Exception as err:
                print(i,end='-')
        return skeptic_volume

    def multi_day_nega_and_vol_greater_avgvol_month (self):
        multi_day_nega_and_vol_greater_avgvol_month=save_filters.copy()
        volume30=self.acc_volume(30)
        stock=list(all_data_price.index)
        for i in stock:
            try:
                multi_day_nega_and_vol_greater_avgvol_month.loc[i,'data']=(MarketWatch.loc[i,'Volume']>volume30.loc[i,'data']) and \
                (all_data_price.loc[i,'data'])['Final'].iloc[-1]<0.97*(all_data_price.loc[i,'data'])['Final'].iloc[-2]<0.97*(all_data_price.loc[i,'data'])['Final'].iloc[-3]\
                <0.97*(all_data_price.loc[i,'data'])['Final'].iloc[-4]
            except Exception as err:
                print(i,end='-')
        return multi_day_nega_and_vol_greater_avgvol_month

    def lowestprice_in_twomonth_highvolume (self):
        minprice=self.minprice(60) 
        lowestprice_in_twomonth_highvolume=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                lowestprice_in_twomonth_highvolume.loc[i,'data']= (minprice.loc[i,'data'] > MarketWatch.loc[i,'Low'] )\
                and MarketWatch.loc[i,'Volume']>=MarketWatch.loc[i,'Base-Vol'] and MarketWatch.loc[i,'Volume']>200000

            except Exception as err:
                print(i,end='-')
        return lowestprice_in_twomonth_highvolume

    def return_from_neg_to_po_scalping (self):
        return_from_neg_to_po_scalping=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                return_from_neg_to_po_scalping.loc[i,'data']=MarketWatch.loc[i,'Close(%)']>-3 and MarketWatch.loc[i,'Final(%)']<-3

            except Exception as err:
                print(i,end='-')
        return return_from_neg_to_po_scalping

    def finnishdownward_and_startupward (self):
        finnishdownward_and_startupward=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                finnishdownward_and_startupward.loc[i,'data']=MarketWatch.loc[i,'Open']<(all_data_price.loc[i,'data'])['Final'].iloc[-2] and \
                MarketWatch.loc[i,'Close(%)']<1 and MarketWatch.loc[i,'No']>10 and MarketWatch.loc[i,'Close']>(all_data_price.loc[i,'data'])['Final'].iloc[-2]

            except Exception as err:
                print(i,end='-')
        return finnishdownward_and_startupward

    def decrease_price_in_month_more20 (self):
        decrease_price_in_month_more20=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                decrease_price_in_month_more20.loc[i,'data']=(((all_data_price.loc[i,'data'])['High'].iloc[-30] - MarketWatch.loc[i,'Close'])*100)/(all_data_price.loc[i,'data'])['High'].iloc[-30]>20


            except Exception as err:
                print(i,end='-')
        return decrease_price_in_month_more20

    def decrease_price_in_month_more30 (self):
        decrease_price_in_month_more30=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                decrease_price_in_month_more30.loc[i,'data']=(((all_data_price.loc[i,'data'])['High'].iloc[-30] - MarketWatch.loc[i,'Close'])*100)/(all_data_price.loc[i,'data'])['High'].iloc[-30]>30


            except Exception as err:
                print(i,end='-')
        return decrease_price_in_month_more30

    def littlestocks_lowest_Base_Vol (self):
        littlestocks_lowest_Base_Vol=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                littlestocks_lowest_Base_Vol.loc[i,'data']==MarketWatch.loc[i,'Open']>=1.02*(all_data_price.loc[i,'data'])['Final'].iloc[-2] and \
                MarketWatch.loc[i,'Final']>=(all_data_price.loc[i,'data'])['Final'].iloc[-2] and \
                ((100*(MarketWatch.loc[i,'High']-MarketWatch.loc[i,'Low']))/MarketWatch.loc[i,'Final'])>2 and \
                MarketWatch.loc[i,'Base-Vol'] <1000000 and MarketWatch.loc[i,'Final(%)'] >0.5


            except Exception as err:
                print(i,end='-')
        return littlestocks_lowest_Base_Vol

    def change_from_pos_to_neg (self):
        change_from_pos_to_neg=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                change_from_pos_to_neg.loc[i,'data']==MarketWatch.loc[i,'Open']>(all_data_price.loc[i,'data'])['Final'].iloc[-2] and \
                MarketWatch.loc[i,'Close']<(all_data_price.loc[i,'data'])['Final'].iloc[-2] 

            except Exception as err:
                print(i,end='-')
        return change_from_pos_to_neg

    def change_from_neg_to_pos (self):
        change_from_neg_to_pos=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                change_from_neg_to_pos.loc[i,'data']==MarketWatch.loc[i,'Open']<(all_data_price.loc[i,'data'])['Final'].iloc[-2] and \
                MarketWatch.loc[i,'Close']>(all_data_price.loc[i,'data'])['Final'].iloc[-2] 

            except Exception as err:
                print(i,end='-')
        return change_from_neg_to_pos

    def Share_No_more50bi_largestocks (self):
        Share_No_more50bi_largestocks=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                Share_No_more50bi_largestocks.loc[i,'data']==MarketWatch.loc[i,'Share-No']>=50000000000

            except Exception as err:
                print(i,end='-')
        return Share_No_more50bi_largestocks

    def Share_No_below100mi_smallstocks (self):
        Share_No_below100mi_smallstocks=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                Share_No_below100mi_smallstocks.loc[i,'data']==MarketWatch.loc[i,'Share-No']<=100000000

            except Exception as err:
                print(i,end='-')
        return Share_No_below100mi_smallstocks

    def large_marketcap(self):
        large_marketcap=self.sort_marketcap()
        large_marketcap=large_marketcap.loc[:,'data']>=0.2
        large_marketcap=pd.DataFrame(large_marketcap)
        return large_marketcap

    def small_marketcap(self):
        small_marketcap=self.sort_marketcap()
        small_marketcap=small_marketcap.loc[:,'data']<=0.2
        small_marketcap=pd.DataFrame(small_marketcap)

        return small_marketcap

    def p_e_below5(self):
        p_e_below5=self.p_E()
        p_e_below5=(p_e_below5['data']>0) & (p_e_below5['data']<=5)
        p_e_below5=pd.DataFrame(p_e_below5)
        return p_e_below5

    def p_e_below0(self):
        p_e_below0=self.p_E()
        p_e_below0=p_e_below0['data']<0
        p_e_below0=pd.DataFrame(p_e_below0)
        return p_e_below0

    def buy_R_greater_sell_R(self):
        buy_R_greater_sell_R=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                buy_R_greater_sell_R.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])>=\
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])

            except Exception as err:
                print(i,end='-')
        return buy_R_greater_sell_R

    def sell_R_greater_buy_R(self):
        sell_R_greater_buy_R=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                sell_R_greater_buy_R.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_R'])/(MarketWatch.loc[i,'No_Buy_R'])<=\
                (MarketWatch.loc[i,'Vol_Sell_R'])/(MarketWatch.loc[i,'No_Sell_R'])

            except Exception as err:
                print(i,end='-')
        return sell_R_greater_buy_R

    def volbuy_I_greater_volbuy_R(self):
        volbuy_I_greater_volbuy_R=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                volbuy_I_greater_volbuy_R.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_I'])>=(MarketWatch.loc[i,'Vol_Buy_R'])

            except Exception as err:
                print(i,end='-')
        return volbuy_I_greater_volbuy_R

    def volbuy_I_greater_volsell_I(self):
        volbuy_I_greater_volsell_I=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                volbuy_I_greater_volsell_I.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_I'])>=(MarketWatch.loc[i,'Vol_Sell_I'])

            except Exception as err:
                print(i,end='-')
        return volbuy_I_greater_volsell_I

    def volsell_I_greater_volsell_R(self):
        volsell_I_greater_volsell_R=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                volsell_I_greater_volsell_R.loc[i,'data']=(MarketWatch.loc[i,'Vol_Sell_I'])>=(MarketWatch.loc[i,'Vol_Sell_R'])

            except Exception as err:
                print(i,end='-')
        return volsell_I_greater_volsell_R

    def volbuy_I_greater_0_3vol(self):
        small_marketcap=self.sort_marketcap()
        small_marketcap=small_marketcap.loc[:,'data']<=0.2
        volbuy_I_greater_0_3vol=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                volbuy_I_greater_0_3vol.loc[i,'data']=(MarketWatch.loc[i,'Vol_Buy_I'])>=0.3*(MarketWatch.loc[i,'Volume'])\
                and (MarketWatch.loc[i,'Vol_Sell_I'])<=(MarketWatch.loc[i,'Vol_Buy_I']) and\
                small_marketcap.loc[i,'data']==True

            except Exception as err:
                print(i,end='-')
        return volbuy_I_greater_0_3vol

    def predict_tomorow_pos(self):
        p_e_pos=self.p_E()
        p_e_pos=(p_e_pos['data']>0)
        predict_tomorow_pos=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                predict_tomorow_pos.loc[i,'data']=(MarketWatch.loc[i,'No'])>50 and \
                (MarketWatch.loc[i,'Base-Vol'])<(MarketWatch.loc[i,'Volume'])\
                and (MarketWatch.loc[i,'Base-Vol'])<=7000000 and\
                (MarketWatch.loc[i,'Close(%)'])>((MarketWatch.loc[i,'Final(%)'])+1.5) and (p_e_pos.loc[i,'data']==True)

            except Exception as err:
                print(i,end='-')
        return predict_tomorow_pos

    def multiday_neg_startreturn_to_pos(self):
        multiday_neg_startreturn_to_pos=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                multiday_neg_startreturn_to_pos.loc[i,'data']=(MarketWatch.loc[i,'Close'])<((MarketWatch.loc[i,'Open'])-(((MarketWatch.loc[i,'Open'])-(MarketWatch.loc[i,'Low']))/2))\
                and (MarketWatch.loc[i,'Close'])>((MarketWatch.loc[i,'Low'])+(((MarketWatch.loc[i,'Open'])-(MarketWatch.loc[i,'Low']))/4))\
                and (MarketWatch.loc[i,'No'])>10 and (MarketWatch.loc[i,'Open'])>(MarketWatch.loc[i,'Low'])\
                and (MarketWatch.loc[i,'Open'])>(all_data_price.loc[i,'data'])['Final'].iloc[-2]\
                or  (MarketWatch.loc[i,'Open'])<(all_data_price.loc[i,'data'])['Final'].iloc[-2]\
                and (MarketWatch.loc[i,'Close(%)'])<1 and (MarketWatch.loc[i,'No'])>10 and\
                    (MarketWatch.loc[i,'Close'])>(all_data_price.loc[i,'data'])['Final'].iloc[-2]\
                or  (MarketWatch.loc[i,'Close'])>1.01*(MarketWatch.loc[i,'Open'])\
                and (MarketWatch.loc[i,'No'])>10 and (MarketWatch.loc[i,'Open'])>1.01*(all_data_price.loc[i,'data'])['Final'].iloc[-2]\
                and (MarketWatch.loc[i,'Close'])!=(MarketWatch.loc[i,'Day_UL'])\
                or  (MarketWatch.loc[i,'Close'])>1.02*(MarketWatch.loc[i,'Open'])\
                and (MarketWatch.loc[i,'No'])>10 and (MarketWatch.loc[i,'Close'])!=(MarketWatch.loc[i,'Day_UL'])\
                or  (MarketWatch.loc[i,'Open'])<1.01*(MarketWatch.loc[i,'Low']) and (MarketWatch.loc[i,'Close(%)'])<=1\
                and (MarketWatch.loc[i,'No'])>10 and (MarketWatch.loc[i,'Close'])>1.02*(MarketWatch.loc[i,'Low'])

            except Exception as err:
                print(i,end='-')
        return multiday_neg_startreturn_to_pos

    def average_vol_greater_base_vol(self):
        volume30=self.acc_volume(30)
        average_vol_greater_base_vol=save_filters.copy()

        stock=list(all_data_price.index)
        for i in stock:
            try:
                average_vol_greater_base_vol.loc[i,'data']=(MarketWatch.loc[i,'Base-Vol']<volume30.loc[i,'data'])

            except Exception as err:
                print(i,end='-')
        return average_vol_greater_base_vol

    def average_vol_less_base_vol(self):
        volume30=self.acc_volume(30)
        average_vol_less_base_vol=save_filters.copy()
        stock=list(all_data_price.index)
        for i in stock:
            try:
                average_vol_less_base_vol.loc[i,'data']=(MarketWatch.loc[i,'Base-Vol']>volume30.loc[i,'data'])

            except Exception as err:
                print(i,end='-')
        return average_vol_less_base_vol


name_filter=['smart_enter_money1','smart_enter_money2','smart_enter_money3',
'I_to_R_smart_enter_money4','I_to_R_smart_enter_money5','I_to_R_smart_enter_money6',
'hot_money1','hot_money2','hot_money3','I_to_R_hot_money1','I_to_R_hot_money2','I_to_R_hot_money3',
'enter_money1','enter_money2','enter_money3','I_to_R_enter_money2','I_to_R_enter_money3',
'I_to_R_vbR_vsI_greater50_bR_greater_sR','I_to_R_vbR_vsI_greater70',
'NbI_greater_2NsI_and_bR_greater_sR',
'smart_exit_money1','smart_exit_money2','smart_exit_money3',
'I_to_R_smart_exit_money1','I_to_R_smart_exit_money2','I_to_R_smart_exit_money3',
'exit_money1','exit_money2','exit_money3',
'I_to_R_exit_money1','I_to_R_exit_money2','I_to_R_exit_money3',
'R_to_I_vbI_vsR_greater50_sR_greater_bR','R_to_I_vbI_vsR_greater70',
'NsI_greater_2NsI_and_sR_greater_bR',
'close_103finally_bR_greater_sR_clockp','pclose_102pfinally_bR_greater_sR_clockp',
'close103_finally_sR_greater_bR_clockn','pclose102_pfinally_sR_greater_bR_clockn',
'skeptic_volume','multi_day_nega_and_vol_greater_avgvol_month','lowestprice_in_twomonth_highvolume',
'return_from_neg_to_po_scalping', 'finnishdownward_and_startupward',
'decrease_price_in_month_more20','decrease_price_in_month_more30','littlestocks_lowest_Base_Vol',
'change_from_pos_to_neg','change_from_neg_to_pos',
'Share_No_more50bi_largestocks','Share_No_below100mi_smallstocks',
'large_marketcap','small_marketcap','p_e_below5','p_e_below0',
'buy_R_greater_sell_R','sell_R_greater_buy_R','volbuy_I_greater_volbuy_R','volbuy_I_greater_volsell_I',
'volsell_I_greater_volsell_R','volbuy_I_greater_0_3vol','predict_tomorow_pos','multiday_neg_startreturn_to_pos',
'average_vol_greater_base_vol','average_vol_less_base_vol']

class bourse_filter(filters):
        item = {index:name_filter[index] for index in range(0,len(name_filter))}
        All = [index for index in range(0,len(name_filter))]
        def get_filters(self,namepositions=All):
            index=[]
            for i in namepositions:
                index .append(f'{i}')
            pos = pd.DataFrame(columns = ['data'],index=index)    
            for i in range(len(namepositions)):
                try:
                    pos.iat[i,0] =eval(f'self.{self.item[namepositions[i]]}()')
                    print('\n',self.item[namepositions[i]],': ',i+1,'\n')
                except Exception as err:
                    print(err)
                    print('*******************************','eror in: ',self.item[namepositions[i]],'--',i,'**************************')
                
            return pos
p=bourse_filter()
d=[index for index in range(0,len(name_filter))]
h=p.get_filters(d)
h
print (time.perf_counter() - start_time, "seconds")

In [ ]:
i=0
j=0
all_filters = pd.DataFrame(columns=h.index , index=bourse_stocks)
for j in bourse_stocks:
    print(j,end='-')
    for i in h.index:
        all_filters.at[j,i]=h.loc[i,'data'].loc[j,'data']
all_filters.columns=name_filter
all_filters

In [ ]:
import pandas as pd
import numpy as np
import os
# abspath = os.path.abspath(__file__)
# dname = os.path.dirname(abspath)
# os.chdir(dname)

class positions:
    strategy=['volsell_I_greater_volsell_R']
    All = bourse_stocks

    def strategy_just(self,all_filters=all_filters,Allpositions=All,strategy=strategy):
        #این تابع فقط اونایی که داخل ماتریس استراتژی هستن و هم زمان شرایطشون درسته را انتخاب میکنه

        w=all_filters.copy()
        c=0
        for i in strategy:
            print(c,end='-')
            for j in bourse_stocks:
                if w.at[j,i]==True:
                       w.at[j,i]=1
            c+=1
        s=[]
        c=0

        for i in strategy:
            print(c,end='-')
            for j in bourse_stocks:
                if w.at[j,i]==1:
                        s.append(j)
            c+=1
        x=list({i for i in s if s.count(i)==len (strategy)})
        stocks=pd.DataFrame(columns=strategy,index=x)
        c=0
        for j in x:
            print(c,end='-')
            for i in strategy:
                stocks.at[j,i]=j
            c+=1
        return w.loc[x],x,stocks

    def strategy_any(self,all_filters=all_filters,Allpositions=All,strategy=strategy):
        # این تابع میاد اونایی رو انتخاب میکنه که شرایط برای اون دو تا صدق کنه . حالا چه هم زمان باشه یا نباشه و در کنارش بقیه استراتژی های بیرون لیست هم دیده میشه

        w=all_filters.copy()
        for i in strategy:
            
            for j in bourse_stocks:
                if w.at[j,i]==True:
                       w.at[j,i]=1


        s=[]
        for i in strategy:
            for j in bourse_stocks:
                if w.at[j,i]==1 :
                    s.append(j)
        
        stocks=pd.DataFrame(columns=strategy,index=s)
        for i in strategy:
            for j in bourse_stocks:
                if w.at[j,i]==1:
                    stocks.at[j,i]=j


        return w.loc[s] , s , stocks
p=positions()
w ,x, stocks=p.strategy_just()
w1 , y , stocks1=p.strategy_any()


In [ ]:
stocks

In [1]:
len(['smart_enter_money1','smart_enter_money2','smart_enter_money3',
'I_to_R_smart_enter_money4','I_to_R_smart_enter_money5','I_to_R_smart_enter_money6',
'hot_money1','hot_money2','hot_money3','I_to_R_hot_money1','I_to_R_hot_money2','I_to_R_hot_money3',
'enter_money1','enter_money2','enter_money3','I_to_R_enter_money2','I_to_R_enter_money3',
'I_to_R_vbR_vsI_greater50_bR_greater_sR','I_to_R_vbR_vsI_greater70',
'NbI_greater_2NsI_and_bR_greater_sR',
'smart_exit_money1','smart_exit_money2','smart_exit_money3',
'I_to_R_smart_exit_money1','I_to_R_smart_exit_money2','I_to_R_smart_exit_money3',
'exit_money1','exit_money2','exit_money3',
'I_to_R_exit_money1','I_to_R_exit_money2','I_to_R_exit_money3',
'R_to_I_vbI_vsR_greater50_sR_greater_bR','R_to_I_vbI_vsR_greater70',
'NsI_greater_2NsI_and_sR_greater_bR',
'close_103finally_bR_greater_sR_clockp','pclose_102pfinally_bR_greater_sR_clockp',
'close103_finally_sR_greater_bR_clockn','pclose102_pfinally_sR_greater_bR_clockn',
'skeptic_volume','multi_day_nega_and_vol_greater_avgvol_month','lowestprice_in_twomonth_highvolume',
'return_from_neg_to_po_scalping', 'finnishdownward_and_startupward',
'decrease_price_in_month_more20','decrease_price_in_month_more30','littlestocks_lowest_Base_Vol',
'change_from_pos_to_neg','change_from_neg_to_pos',
'Share_No_more50bi_largestocks','Share_No_below100mi_smallstocks',
'large_marketcap','small_marketcap','p_e_below5','p_e_below0',
'buy_R_greater_sell_R','sell_R_greater_buy_R','volbuy_I_greater_volbuy_R','volbuy_I_greater_volsell_I',
'volsell_I_greater_volsell_R','volbuy_I_greater_0_3vol','predict_tomorow_pos','multiday_neg_startreturn_to_pos',
'average_vol_greater_base_vol','average_vol_less_base_vol'])

65